In [7]:
import sys
sys.path.append("../")
import torch

# Tokenize
from src.components.bpe_tokenizer import BPETokenizer
# Input Embeddings (Just the combination of TokenEmbedding and PositionalEncoding)
from src.components.input_embeddings import InputEmbeddings

**1. Parameters**

In [8]:
batch_size = 2
seq_len = 10
vocab_size = 300
d_model = 128
max_seq_len = 512

**2. Train the Tokenizer**

In [9]:
sample = """
The quick brown fox jumps over the lazy dog.
Machine learning is fascinating and powerful.
Transformers have revolutionized natural language processing.
"""

tokenizer = BPETokenizer(vocab_size=vocab_size)
tokenizer.train(sample)

2026-02-03 11:09:11.424 | INFO     | src.components.bpe_tokenizer:train:63 - Training BPE with target vocab size: 300
2026-02-03 11:09:11.425 | INFO     | src.components.bpe_tokenizer:train:67 - Found 28 word pieces
2026-02-03 11:09:11.427 | SUCCESS  | src.components.bpe_tokenizer:train:106 - 
Final vocab size: 302


**3. Encode the text**

In [10]:
sentence1 = "The fox jumps"
sentence2 = "Machine learning"

encoded1 = tokenizer.encode(sentence1) 
encoded2 = tokenizer.encode(sentence2)
print(encoded1)
print(encoded2)

[266, 277, 282]
[298, 299, 97, 114, 110, 259]


**4. Padding/truncating to fixed length**

*When using batching, we need to pad/truncate the sequences to a fixed length.*

In [ ]:
# 4. Pad to same length (seq_len=10) for batching
# Pad with 0 (or use your <|pad|> token)
padded1 = encoded1 + [0] * (seq_len - len(encoded1))
padded2 = encoded2 + [0] * (seq_len - len(encoded2))

# Truncate if too long
padded1 = padded1[:seq_len]
padded2 = padded2[:seq_len]

print(padded1)
print(padded2)

# x = torch.randint(0, vocab_size, (batch_size, seq_len))
x = torch.tensor([padded1, padded2])
print(x.shape)

[266, 277, 282, 0, 0, 0, 0, 0, 0, 0]
[298, 299, 97, 114, 110, 259, 0, 0, 0, 0]
torch.Size([2, 10])


**5. Generate the final embedding**

In [13]:
emb = InputEmbeddings(vocab_size, d_model, max_seq_len, dropout=0.1)
out = emb(x)

2026-02-03 11:09:33.816 | DEBUG    | src.components.token_embedding:__init__:24 - TokenEmbedding: vocab=300, dim=128
2026-02-03 11:09:33.819 | DEBUG    | src.components.positional_encoding:__init__:54 - SinusoidalPositionalEncoding: max_len=512, dim=128, dropout=0.1
2026-02-03 11:09:33.821 | DEBUG    | src.components.input_embeddings:__init__:31 - InputEmbeddings: vocab=300, dim=128, max_len=512, dropout=0.1


In [14]:
x.shape

torch.Size([2, 10])

---